# Data Cleaning

In [137]:
import pandas as pd

## Firm data

In [138]:
firm_data = pd.read_csv('./data/dirty/bds2021_fa.csv')

In [139]:
firm_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   year    528 non-null    int64 
 1   fage    528 non-null    object
 2   firms   528 non-null    object
 3   estabs  528 non-null    object
 4   emp     528 non-null    object
dtypes: int64(1), object(4)
memory usage: 20.8+ KB


In [140]:
firm_data.head()

,year,fage,firms,estabs,emp
0,1978,a) 0,486456,493546,2545510
1,1978,b) 1,355229,365696,2345414
2,1978,c) 2,X,X,X
3,1978,d) 3,X,X,X
4,1978,e) 4,X,X,X


### Cleaning the firm age column

In [141]:
firm_data['fage'].unique()

array(['a) 0', 'b) 1', 'c) 2', 'd) 3', 'e) 4', 'f) 5', 'g) 6 to 10',
       'h) 11 to 15', 'i) 16 to 20', 'j) 21 to 25', 'k) 26+',
       'l) Left Censored'], dtype=object)

In [142]:
def extract_age(x):
    try:
        age = x.split(' ')[1:]
        
        if len(age) == 1:
            return age[0]
        
        age = ' '.join(age)
        age = age.replace(' to ', '-')
        
        return age
    except:
        return None
    
firm_data['fage'] = firm_data['fage'].apply(extract_age)

In [143]:
firm_data['fage'].unique()

array(['0', '1', '2', '3', '4', '5', '6-10', '11-15', '16-20', '21-25',
       '26+', 'Left Censored'], dtype=object)

In [144]:
# drop all rows that include 'X'
firm_data = firm_data[firm_data['firms'] != 'X']

In [145]:
firm_data.to_csv('./data/clean/firm_data.csv', index=False)

## Aggregate data

In [146]:
aggregate = pd.read_csv('./data/dirty/bds2021_aggregate.csv')

### Create firm birth and death rates

In [147]:
df = firm_data[firm_data['fage'] == '0'].copy()
df.rename(columns={'firms': 'firmbirth_firms', 'estabs': 'firmbirth_estabs', 'emp': 'firmbirth_emp'}, inplace=True)
df.drop(columns=['fage'], inplace=True)

In [148]:
aggregate = pd.merge(aggregate, df, on='year', how='inner')

In [149]:
# convert all columns to numeric
for col in aggregate.columns:
    if col != 'year':
        aggregate[col] = pd.to_numeric(aggregate[col])

In [150]:
aggregate['firmbirth_rate'] = aggregate['firmbirth_firms'] / aggregate['firms']
aggregate['firmdeath_rate'] = aggregate['firmdeath_firms'] / aggregate['firms']

In [151]:
aggregate.to_csv('./data/clean/aggregate.csv', index=False)

## R&D

In [152]:
real_rnd = pd.read_csv('./data/dirty/real_r&d.csv')

real_rnd.rename(columns={'Y694RX1A020NBEA': 'rnd', 'observation_date': 'year'}, inplace=True)

In [153]:
patents = pd.read_csv('./data/dirty/patents.csv')

patents.rename(columns={'PATENTUSALLTOTAL': 'patents', 'observation_date': 'year'}, inplace=True)

pd.merge(patents, real_rnd, on='year', how='outer').to_csv('./data/clean/real_rnd_patents.csv', index=False)